## Toronto Neighborhoods
### Andrew Rodgers
Import all the necessary modules and make scrape the data from the webpage

In [226]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import time
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

resp = requests.get(url=url)

soup = BeautifulSoup(resp.text,"html.parser")

Format the html to list, one for the header and another for the data

In [261]:
header = []
data = []
for i in table.findAll('tr'):
    for j in i.findAll('th'):
        header.append(j.text)
    for j in i.findAll('td'):
        data.append(j.text)

Format the header and data lists into a *pandas* dataframe, **df**

In [261]:
header[2] = header[2][0:-1]
col1 = []
col2 = []
col3 = []
for i in range(0,len(data),3):
    col1.append(data[i])
    col2.append(data[i+1])
    col3.append(data[i+2])
    
df = pd.DataFrame(list(zip(col1,col2,col3)),columns = header)

Subset **df** so we only use the rows with assigned Boroughs  
Use the Borough for the neighborhood name if not assigned

In [261]:
df = df[df.Borough != 'Not assigned']

for i in range(0,len(df['Postcode'])):
    if "\n" in df.iloc[i,2]:
        df.iloc[i,2] = df.iloc[i,2][0:-1]
    if df.iloc[i,2] == "Not assigned":
        df.iloc[i,2] = df.iloc[i,1]

Sort **df** by Postcode to make searchign easier  
Loop through **df** to combine neighborhoods in the same cell, separated by commas

In [271]:
df = df.sort_values('Postcode',0)

for i in range(0,len(df['Postcode'])):
    if df.iloc[i-1,0] == df.iloc[i,0]:
        #if df.iloc[i-1,2] in df.iloc[i,2]:
        df.iloc[i,2] = df.iloc[i-1,2] + ", " + df.iloc[i,2]
        
df = df.drop_duplicates(subset='Postcode', keep='last', inplace=False)

Here's a sample of **df**

In [273]:
df.drop_duplicates(subset='Borough')

,Postcode,Borough,Neighbourhood
12,M1B,Scarborough,"Rouge, Rouge, Malvern"
63,M2H,North York,Hillcrest Village
16,M4B,East York,"Woodbine Gardens, Woodbine Gardens, Parkview Hill"
47,M4E,East Toronto,The Beaches
144,M4N,Central Toronto,Lawrence Park
239,M4W,Downtown Toronto,Rosedale
35,M6C,York,Humewood-Cedarvale
72,M6H,West Toronto,"Dovercourt Village, Dovercourt Village, Dufferin"
8,M7A,Queen's Park,Queen's Park
174,M7R,Mississauga,Canada Post Gateway Processing Centre


In [275]:
df.shape

(103, 3)